<a href="https://colab.research.google.com/github/keerthan-smart/Image-classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bing-image-downloader


In [ ]:
!mkdir images


In [ ]:
from bing_image_downloader import downloader
downloader.download("pretty sunflower",limit=30,output_dir='images',adult_filter_off=True)
downloader.download("rugby ball leather",limit=30,output_dir='images',adult_filter_off=True)
downloader.download("ice cream cone",limit=30,output_dir='images',adult_filter_off=True)


In [6]:
# Preprocessing
import os 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize

target = []
images = []
flat_data = []

DATADIR = '/content/images'
CATEGORIES = ['pretty sunflower','rugby ball leather','ice cream cone']

for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [ ]:
df = pd.DataFrame(flat_data)
df['Target'] = target
df

In [9]:
#Splitting data into Training and Testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.3,random_state=109)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
parameters = [
             {'C':[1,10,100,1000],'kernel':['linear']},
             {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]
svc = svm.SVC(probability=True)
cv = GridSearchCV(svc,parameters)
cv.fit(x_train,y_train)

In [ ]:
y_pred = cv.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
confusion_matrix(y_pred,y_test)

In [ ]:
accuracy_score(y_pred,y_test)


In [ ]:
print(classification_report(y_pred,y_test))

In [19]:
#Save the model using Pickle library
import pickle
pickle.dump(cv,open('img_model.p','wb'))

In [20]:
model = pickle.load(open('img_model.p','rb'))


In [ ]:
#Testing a new image
flat_data = []
url = input('Enter the URL')
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = CATEGORIES[y_out[0]]
print(f' PREDICTED OUTPUT: {y_out}')